In [3]:
import arcpy
from arcpy.sa import *
import requests
import zipfile
import os
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind

In [ ]:
#ice velocity data is a .nc file, convert to raster
icevel_nc = r"C:\Users\\schil726\Downloads\GIS5571_FinalProj\data\Antarctica_ice_velocity_2019_2020_1km_v01.1.nc"

#velocity is in x direction and y direction. need to get layers for both
arcpy.md.MakeNetCDFRasterLayer(
in_netCDF_file = icevel_nc,
variable = "VX",
x_dimension = "x",
y_dimension = "y")

arcpy.md.MakeNetCDFRasterLayer(
in_netCDF_file = icevel_nc,
variable = "VY",
x_dimension = "x",
y_dimension = "y")

In [ ]:
#need to calculate magnitude from x and y velocity
vx_raster = Raster(r"C:\Users\schil726\Downloads\GIS5571_FinalProj\data\ice_VX.tif")
vy_raster = Raster(r"C:\Users\schil726\Downloads\GIS5571_FinalProj\data\ice_VY.tif")

# Calculate speed (magnitude)
magnitude_raster = SquareRoot(Power(vx_raster, 2) + Power(vy_raster, 2))

magnitude_raster.save(r"C:\Users\schil726\Downloads\GIS5571_FinalProj\data\ice_magnitude.tif")

In [ ]:
#bring in dem
DEM = Raster(r"C:\Users\schil726\Downloads\GIS5571_FinalProj\data\PineIsland_REMA_mosaics\mosaic_dem.tif")

#bring in bedmap2 bed topography
bedmap_bedtopo = Raster(r"C:\Users\schil726\Downloads\GIS5571_FinalProj\data\bedmap2_tiff\bedmap2_tiff\bedmap2_bed.tif")

In [8]:
#all 3 rasters are in. We will use the DEM mosaic as a mask to remove unecessary data
def clip_by_mask(raster_layers, mask, output_path):
        for raster in raster_layers:
            out_extract = arcpy.sa.ExtractByMask(
                in_raster = raster,
                in_mask_data = mask,
                extraction_area = "INSIDE"
                )
            out_extract.save(f"{output_dir}/{raster}_clipped.tif")
        
raster_layers = ["bedmap2_bed.tif", "ice_magnitude.tif"]
mask = "mosaic_dem.tif"
output_dir = r"C:\Users\schil726\Downloads\GIS5571_FinalProj\data\extracted"

clip_by_mask(raster_layers, mask, output_dir)

In [9]:
#create slope from surface DEM
arcpy.ddd.Slope(
    in_raster = "mosaic_dem.tif",
    out_raster = "REMA_slope",
)

<Result 'C:\\Users\\schil726\\Downloads\\GIS5571_FinalProj\\GIS\\GIS\\Default.gdb\\REMA_slope'>

In [10]:
#create slope from bed topo
arcpy.ddd.Slope(
    in_raster = "bedmap2_bed.tif_clipped.tif",
    out_raster = "bedmap_slope",
)

<Result 'C:\\Users\\schil726\\Downloads\\GIS5571_FinalProj\\GIS\\GIS\\Default.gdb\\bedmap_slope'>

In [14]:
#lets try a fuzzy reclass
#input raster
in_raster = "REMA_slope"

#create FuzzyLinear algorithm object
min = 0.001
max = 88.93
REMAFuzzyAlgorithm = FuzzyLinear(min,max)

REMA_fuzzy_slope = FuzzyMembership(
    in_raster,
    REMAFuzzyAlgorithm
)

In [15]:
#quick - lets compare with regular reclass
arcpy.ddd.Reclassify(
    in_raster = "REMA_slope",
    reclass_field = "VALUE",
    remap = "0 1.593 1;1.594 3.287 2;3.288 4.981 3;4.982 88.93 4",
    out_raster = r"C:\Users\schil726\Downloads\GIS5571_FinalProj\data\reclass\REMA_reclass.tif",
)

<Result 'C:\\Users\\schil726\\Downloads\\GIS5571_FinalProj\\data\\reclass\\REMA_reclass.tif'>

In [17]:
#fuzzy reclass looks better
#fuzzy reclass for bedmap topo

#input raster
in_raster = "bedmap_slope"

#create FuzzyLinear algorithm object
min = 0.001
max = 15.359
bedmapFuzzyAlgorithm = FuzzyLinear(min,max)

bedmap_fuzzy_slope = FuzzyMembership(
    in_raster,
    bedmapFuzzyAlgorithm
)

In [18]:
#fuzzy reclass for ice magnitude
#input raster
in_raster = "ice_magnitude.tif_clipped.tif"

#create FuzzyLinear algorithm object
min = 0.041
max = 5642.869
velocityFuzzyAlgorithm = FuzzyLinear(min,max)

icevel_fuzzy_slope = FuzzyMembership(
    in_raster,
    velocityFuzzyAlgorithm
)

In [21]:
#weight my layers
weight_REMA = 0.4
weight_icevel = 0.4
weight_bedmap = 0.2

#define inputs
REMA_raster = "REMA_fuzzy_slope"
icevel_raster = "icevel_fuzzy_slope"
bedmap_raster = "bedmap_fuzzy_slope"

#create cost surface raster
weighted_costsurface_multiply = (
    (arcpy.sa.Raster(REMA_raster)*weight_REMA) *
    (arcpy.sa.Raster(icevel_raster)*weight_icevel) *
    (arcpy.sa.Raster(bedmap_raster)*weight_bedmap )
)

output_costsurface_multiply_raster = r"C:\Users\schil726\Downloads\GIS5571_FinalProj\data\costsurface\costsurface_multiply_raster.tif"
weighted_costsurface_multiply.save(output_costsurface_multiply_raster)

In [25]:
#weight my layers
weight_REMA = 0.4
weight_icevel = 0.4
weight_bedmap = 0.2

#define inputs
REMA_raster = "REMA_fuzzy_slope"
icevel_raster = "icevel_fuzzy_slope"
bedmap_raster = "bedmap_fuzzy_slope"

#create cost surface raster (this time add!)
weighted_costsurface_add = (
    (arcpy.sa.Raster(REMA_raster)*weight_REMA) +
    (arcpy.sa.Raster(icevel_raster)*weight_icevel) +
    (arcpy.sa.Raster(bedmap_raster)*weight_bedmap )
)

output_costsurface_add_raster = r"C:\Users\schil726\Downloads\GIS5571_FinalProj\data\costsurface\costsurface_add_raster.tif"
weighted_costsurface_add.save(output_costsurface_add_raster)

In [26]:
#cost surface is a bit too generalized and influenced by bed topo with addition
#i prefer multiplication, but let's try with even weights
#weight my layers
weight_REMA = 0.33
weight_icevel = 0.33
weight_bedmap = 0.33

#define inputs
REMA_raster = "REMA_fuzzy_slope"
icevel_raster = "icevel_fuzzy_slope"
bedmap_raster = "bedmap_fuzzy_slope"

#create cost surface raster (this time add!)
evenweighted_costsurface = (
    (arcpy.sa.Raster(REMA_raster)*weight_REMA) *
    (arcpy.sa.Raster(icevel_raster)*weight_icevel) *
    (arcpy.sa.Raster(bedmap_raster)*weight_bedmap )
)

output_costsurface_raster = r"C:\Users\schil726\Downloads\GIS5571_FinalProj\data\costsurface\costsurface_evenweight_raster.tif"
evenweighted_costsurface.save(output_costsurface_raster)

In [12]:
#assign cost surface vales to crevasse yes/no points
arcpy.sa.ExtractValuesToPoints(
    in_point_features="verification_points",
    in_raster="weighted_costsurface_multiply",
    out_point_features="crevasse_points_with_values",
    interpolate_values="NONE"
)

<geoprocessing server result object object at 0x0000016588ABBE10>

In [13]:
#export to CSV
arcpy.conversion.TableToTable(
    "crevasse_points_with_values",
    r"C:\Users\schil726\Downloads\GIS5571_FinalProj\verification",
    "crevasse_verification.csv"
)

<Result 'C:\\Users\\schil726\\Downloads\\GIS5571_FinalProj\\verification\\crevasse_verification.csv'>

In [14]:
#use scipy.stats for t-test
data = pd.read_csv(r"C:\Users\schil726\Downloads\GIS5571_FinalProj\verification\crevasse_verification.csv")

#separate cost values for yes/no
crevasse_yes = data[data['Crevasse'] == "Yes"]['RASTERVALU']
crevasse_no = data[data['Crevasse'] == "No"]['RASTERVALU']

#perform t-test
t_stat, p_value = ttest_ind(crevasse_yes, crevasse_no, equal_var=False)
print(f"T-Statistic: {t_stat}, P-Value: {p_value}")

T-Statistic: 3.9500922364802236, P-Value: 0.000175684610102458
